<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:


      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-08 14:48:29
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: -25.59 K (-0.19%)
Current PnL: -13.37 L (-9.32%)
CY Booked + Current PnL: -6.41 L (-4.47%)
-------------------
Total profit:  4.65 L
Total loss:  -18.02 L
-------------------
Total Booked + Current PnL: 20.57 L (17.65%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.09%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.88 L (52.56%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.78%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,6904.43,8285.00,-16.63,H-LC,91.67,113632.0,10066.0,10647.0,-0.53,9.72,9.37,20.00,32.0,0.95,0.83,25.16,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-20.73,H-LC,83.33,109408.0,5096.0,12440.0,-1.20,4.89,11.37,16.81,27.0,0.41,0.80,21.72,X5K,ATH,METALS
71,SBIN,760.30,863.00,-11.51,M-LC,33.33,214451.0,12972.0,14240.0,0.30,6.44,6.64,13.51,77.0,0.91,1.57,19.85,XY25,NTT,BANKS
92,VIPIND,488.80,489.00,-212.13,H-SC,98.96,80886.0,-14430.0,14471.0,-0.88,-15.14,17.89,0.04,157.0,-1.00,0.59,59.54,OX40N,NTT,MISC
88,UNIONBANK,123.87,163.00,-2.93,M-LC,18.75,170300.0,29460.0,15037.0,-1.57,20.92,8.83,31.59,88.0,1.96,1.25,47.93,XY24,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-31.29,M-MC,7.29,179485.0,-323.0,112735.0,-0.87,-0.18,62.81,62.52,173.0,-0.00,1.31,33.85,XR,NTT,BANKS
38,HEROMOTOCO,4311.81,6039.03,-6.67,H-MC,29.17,150017.0,-896.0,61342.0,-0.48,-0.59,40.89,40.06,103.0,-0.01,1.10,22.70,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-50.15,H-MC,58.33,100642.0,138.0,30505.0,0.71,0.14,30.31,30.49,97.0,0.00,0.74,32.59,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VALIANTORG,512.64,838.00,-363.88,H-SC,28.12,112294.0,-20992.0,105590.0,3.55,-15.75,94.03,63.47,151.0,-0.20,0.82,88.23,XR,NTT,CHEMICALS
85,TRIDENT,37.35,48.00,26.18,M-SC,78.12,79361.0,-12894.0,39196.0,3.25,-13.98,49.39,28.51,233.0,-0.33,0.58,35.80,XR,NTT,TEXTILES
16,BLUSPRING,226.45,84.55,NaN,NaN,39.06,18418.0,-31175.0,99.0,2.98,-62.86,0.54,-62.66,270.0,-314.90,0.13,23.51,XY24,ATH,MISC
13,BATAINDIA,1550.24,2096.00,-30.58,M-SC,68.75,103866.0,-24804.0,70099.0,1.88,-19.28,67.49,35.20,205.0,-0.35,0.76,5.97,X40,NTT,FOOTWEAR
69,SAMMAANCAP,170.35,326.00,-165.91,M-SC,37.50,75666.0,-26544.0,119931.0,1.79,-25.97,158.50,91.37,212.0,-0.22,0.55,22.83,XY25,NTT,FINANCE


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
94,WHIRLPOOL,1167.49,2270.00,-40.45,M-SC,31.25,196333.0,24712.0,137355.0,-2.71,14.40,69.96,94.43,201.0,0.18,1.44,44.91,XR,NTT,DURABLES
24,COFFEEDAY,59.14,80.00,-57.75,L-SC,52.08,63821.0,-49728.0,89777.0,-2.46,-43.79,140.67,35.27,265.0,-0.55,0.47,56.20,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,8.27,X-SC,97.92,100697.0,-266.0,94796.0,-2.34,-0.26,94.14,93.63,67.0,-0.00,0.74,49.99,SR,ATH,CHEMICALS
65,RELAXO,902.64,1176.00,-29.89,H-SC,95.83,63424.0,-52114.0,87107.0,-2.14,-45.11,137.34,30.28,131.0,-0.60,0.46,23.91,X40N,NTT,FOOTWEAR
56,LAOPALA,369.40,464.00,116.69,H-SC,67.19,68946.0,-31900.0,57728.0,-1.90,-31.63,83.73,25.61,136.0,-0.55,0.50,24.35,AR,NTT,CERAMICS


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,47.02,M-SC,22.92,127332.0,-15147.0,15764.0,0.34,-10.63,12.38,0.44,247.0,-0.96,0.93,50.29,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-28.99,M-SC,25.52,151410.0,-19731.0,19683.0,0.37,-11.53,13.00,-0.03,191.0,-1.00,1.11,8.82,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,92.71,148936.0,-25623.0,25662.0,0.02,-14.68,17.23,0.02,133.0,-1.00,1.09,29.15,OX40N,NTT,PAINTS
92,VIPIND,488.80,489.0,-212.13,H-SC,98.96,80886.0,-14430.0,14471.0,-0.88,-15.14,17.89,0.04,157.0,-1.00,0.59,59.54,OX40N,NTT,MISC
81,TATAELXSI,7332.28,7332.0,-17.98,X-MC,9.38,86058.0,-16594.0,16592.0,-0.45,-16.17,19.28,-0.00,59.0,-1.00,0.63,30.82,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-18.38,X-LC,39.06,265387.0,-30547.0,124467.0,-0.27,-10.32,46.90,31.73,1.0,-0.25,1.94,6.22,X40,BTT,IT
48,INFY,1461.46,2275.00,-8.83,X-LC,77.08,292629.0,31028.0,114594.0,0.48,11.86,39.16,55.67,2.0,0.27,2.14,18.69,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-14.44,X-LC,56.77,267814.0,-2533.0,37173.0,-0.80,-0.94,13.88,12.81,5.0,-0.07,1.96,12.01,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.04,X-LC,65.62,268479.0,39246.0,28029.0,-0.81,17.12,10.44,29.35,10.0,1.40,1.96,27.57,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-2.64,X-LC,23.96,82358.0,-27814.0,116891.0,0.49,-25.25,141.93,80.85,11.0,-0.24,0.60,21.30,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3041.00,NaN,NaN,94.79,76025.0,41576.0,0.0,1.36,120.69,0.00,120.69,274.0,inf,0.56,16.26,AR,ATH,ELECTRICAL
28,DIGITIDE,188.38,237.72,NaN,NaN,34.38,50363.0,9108.0,1697.0,0.64,22.08,3.37,26.19,272.0,5.37,0.37,18.57,XY24,ATH,IT
52,JPPOWER,18.73,26.20,-14.84,L-SC,100.00,171377.0,29535.0,27043.0,-0.09,20.82,15.78,39.88,262.0,1.09,1.25,73.68,XY24,NTT,POWER
88,UNIONBANK,123.87,163.00,-2.93,M-LC,18.75,170300.0,29460.0,15037.0,-1.57,20.92,8.83,31.59,88.0,1.96,1.25,47.93,XY24,NTT,BANKS


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,92.75,M-MC,51.04,231626.0,6664.0,160980.0,0.78,2.96,69.50,74.53,189.0,0.04,1.69,34.48,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.64,24.70,H-SC,32.29,127663.0,2366.0,56708.0,-1.24,1.89,44.42,47.15,143.0,0.04,0.93,59.71,MH,ATH,POWER
20,CAMS,3643.00,5250.99,2.76,H-SC,19.79,115419.0,13415.0,31613.0,-0.48,13.15,27.39,44.14,126.0,0.42,0.84,31.53,X40N,ATH,MISC
95,WIPRO,243.46,420.00,-2.86,M-LC,88.54,166532.0,15587.0,93874.0,0.32,10.33,56.37,72.51,71.0,0.17,1.22,14.71,XR,NTT,IT
88,UNIONBANK,123.87,163.00,-2.93,M-LC,18.75,170300.0,29460.0,15037.0,-1.57,20.92,8.83,31.59,88.0,1.96,1.25,47.93,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3041.00,NaN,NaN,94.79,76025.0,41576.0,0.0,1.36,120.69,0.00,120.69,274.0,inf,0.56,16.26,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-40.45,M-SC,31.25,196333.0,24712.0,137355.0,-2.71,14.40,69.96,94.43,201.0,0.18,1.44,44.91,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.86,M-LC,88.54,166532.0,15587.0,93874.0,0.32,10.33,56.37,72.51,71.0,0.17,1.22,14.71,XR,NTT,IT
45,INDIAMART,2327.09,4911.36,-48.08,H-SC,70.83,135325.0,11989.0,124973.0,0.59,9.72,92.35,111.05,123.0,0.10,0.99,32.59,AR,ATH,MISC
54,KPIGREEN,497.21,731.64,24.70,H-SC,32.29,127663.0,2366.0,56708.0,-1.24,1.89,44.42,47.15,143.0,0.04,0.93,59.71,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,39.06,18418.0,-31175.0,99.0,2.98,-62.86,0.54,-62.66,270.0,-314.90,0.13,23.51,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-57.75,L-SC,52.08,63821.0,-49728.0,89777.0,-2.46,-43.79,140.67,35.27,265.0,-0.55,0.47,56.20,XR,NTT,HOTELS
32,ENRIN,1377.95,3041.00,NaN,NaN,94.79,76025.0,41576.0,0.0,1.36,120.69,0.00,120.69,274.0,inf,0.56,16.26,AR,ATH,ELECTRICAL
58,MASFIN,326.60,399.50,-17.80,H-SC,69.79,93420.0,-4560.0,26429.0,0.73,-4.65,28.29,22.32,145.0,-0.17,0.68,35.49,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-1.25,X-LC,42.71,98510.0,765.0,36636.0,0.20,0.78,37.19,38.26,18.0,0.02,0.72,31.54,X40,BTT,FINANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-14.84,L-SC,100.00,171377.0,29535.0,27043.0,-0.09,20.82,15.78,39.88,262.0,1.09,1.25,73.68,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-212.13,H-SC,98.96,80886.0,-14430.0,14471.0,-0.88,-15.14,17.89,0.04,157.0,-1.00,0.59,59.54,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,8.27,X-SC,97.92,100697.0,-266.0,94796.0,-2.34,-0.26,94.14,93.63,67.0,-0.00,0.74,49.99,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-3.59,H-SC,96.88,180730.0,16060.0,53026.0,-1.06,9.75,29.34,41.95,124.0,0.30,1.32,17.95,X40N,ATH,AC
65,RELAXO,902.64,1176.00,-29.89,H-SC,95.83,63424.0,-52114.0,87107.0,-2.14,-45.11,137.34,30.28,131.0,-0.60,0.46,23.91,X40N,NTT,FOOTWEAR


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.92
1,25,41.84
2,50,70.54


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.45
LC    33.93
MC    21.54
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.92
X40      12.96
XY25     12.05
X40N     11.92
XR       10.02
OX40N     8.19
AR        8.17
MH        1.67
X5K       1.44
X200      1.41
SR        1.23
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.45
M-SC    15.89
H-LC    14.11
X-LC    12.90
H-MC     9.53
X-MC     7.60
M-LC     5.88
X-SC     4.46
M-MC     4.06
L-SC     2.65
L-LC     1.04
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.19,-0.83,34.61
IT,12.12,-9.67,63.53
FINANCE,8.51,-11.96,52.58
BANKS,7.68,-8.56,52.86
MISC,5.83,-20.68,77.23
PAINTS,5.71,-12.05,34.13
ELECTRICAL,5.46,1.27,45.06
HEALTHCARE,4.65,-1.55,31.21
AUTO,4.31,-14.82,65.30


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2665041.0,26
XR,1087882.0,13
AR,1031852.0,10
X40,684060.0,10
X40N,533413.0,11
XY25,473107.0,8
OX40N,308985.0,11
SR,176956.0,2
MH,87213.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1988413.0,20
M-SC,1770294.0,21
H-LC,611233.0,12
X-LC,546265.0,8
H-MC,532654.0,8
X-MC,415673.0,6
M-MC,414639.0,3
X-SC,309401.0,5
M-LC,258678.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      892140.0      6
M-SC       XY24      810152.0      7
H-SC       AR        528594.0      3
M-SC       XR        339463.0      4
M-MC       XY24      301904.0      2
H-MC       XY24      301532.0      3
X-LC       X40       275697.0      3
H-SC       XR        213259.0      3
           X40N      208564.0      4
H-LC       X40N      195877.0      4
M-SC       AR        185365.0      2
           OX40N     163124.0      5
X-MC       X40       156730.0      2
H-LC       X40       152938.0      3
L-SC       XR        151737.0      2
M-LC       XY24      150564.0      2
X-SC       XR        122633.0      1
M-SC       XY25      119931.0      1
X-LC       AR        116891.0      1
M-MC       XR        112735.0      1
X-LC       XY25      105233.0      3
X-MC       XY25      103132.0      1
X-SC       SR         94796.0      1
M-LC       XR         93874.0      1
H-SC       OX40N      89148.0      3
X-MC       XY24       86754.0      1
H-MC       XY25       82616.0      1
M-SC       SR         82160.0      1
H-LC       AR         82026.0      1
           X5K        71304.0      2
M-SC       X40        70099.0      1
H-LC       X200       67783.0      1
H-MC       AR         61342.0      1
L-SC       AR         57634.0      1
H-SC       MH         56708.0      1
L-MC       XR         54181.0      1
X-MC       X40N       52465.0      1
X-SC       XY24       51851.0      1
X-LC       X40N       48444.0      1
L-LC       XY25       47955.0      1
H-LC       XY24       41305.0      1
X-SC       OX40N      40121.0      2
H-MC       MH         30505.0      1
           X40        28596.0      1
           X40N       28063.0      1
L-SC       XY24       27043.0      1
X-MC       OX40N      16592.0      1
M-LC       XY25       14240.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
